# Data Cleaning

This script loads raw data from the Building-Data-Genome-Project-2 and performs data Cleaning.

In [27]:
# Import modules
import pandas as pd
import numpy as np

# Parameter Selection
meter_data = ["electricity", "gas", "hotwater", "chilledwater"]

# Path & url definition
url_root = 'https://media.githubusercontent.com/media/buds-lab/building-data-genome-project-2/master/data/'
path_meters = "meters/raw/"
path_data_out = "..\\data\\"
path_fig_out = "..\\figures\\"

meter_files = [url_root + path_meters + meter+ ".csv" for meter in meter_data]

## Functions

In [28]:
def cumulativemissing_id(df_in, p_freq='H', cumulative_threshold=4):
    """"Function to identify sections of cumulative missing values in a dataframe."""

    t_end, t_start = dict(), dict()
    for col in df_in:
        # Count number of consecutive NaNs in dataframe
        # Source: https://stackoverflow.com/questions/29007830/identifying-consecutive-nans-with-pandas
        df_NaNcount = pd.concat([df_in[col],
                                 (df_in[col].isnull().astype(int)
                                  .groupby(df_in[col].notnull().astype(int).cumsum())
                                  .cumsum().to_frame('consec_count'))], axis=1)
        # Loop over the cumulative missing values violating the threshold
        t_end[col] = []
        t_start[col] = []
        t_MinusOne = 0
        for t in df_NaNcount['consec_count'][df_NaNcount['consec_count'] >= cumulative_threshold].index:
            # If next value of the ones exceeding the threshold == 0 then this one is the maximal one
            try:
                if df_NaNcount['consec_count'][t + pd.to_timedelta(1, unit=p_freq)] == 0:
                    t_end[col].append(t)  # identified timestep of max cumulative missing values
                    max_CumNaN = df_NaNcount['consec_count'][t]  # max cumulative missing values
                    t_begin = t - pd.to_timedelta(max_CumNaN - 1,
                                                  unit=p_freq)  # identified first timestep of cumulative missing values
                    t_start[col].append(t_begin)  # Store values of t_start
                    # df_NaNcount.loc[t_begin:t, 'consec_count'] = [max_CumNaN] * max_CumNaN  # Replacing NaN counts with max cumulated value
            # If next value of the ones exceeding the threshold cannot be obtained (end of the dataframe) then the previous one is the maximum one
            except:
                t_end[col].append(t_MinusOne)  # identified timestep of max cumulative missing values
                max_CumNaN = df_NaNcount['consec_count'][t_MinusOne]  # max cumulative missing values
                t_begin = t_MinusOne - pd.to_timedelta(max_CumNaN - 1,
                                              unit=p_freq)  # identified first timestep of cumulative missing values
                t_start[col].append(t_begin)  # Store values of t_start
            t_MinusOne = t
    return t_start, t_end


def gap_fill(df_in, t_start, t_end, p_freq='H'):
    """"Hot-deck function - filling identified gaps from averaged surrounding week values."""

    # Define the time intervals where to drop data
    for col in t_start:
        itv_tot = pd.date_range(start=df_in[col].index[0], end=df_in[col].index[-1], freq=p_freq)
        for i in range(len(t_start[col])):
            shift_before_break = False
            error = False
            t_itv = pd.date_range(start=t_start[col][i], end=t_end[col][i], freq=p_freq)
            # Identify the two time intervals before and after the missing values
            nb_days = (t_end[col][i] - t_start[col][i]).days
            shift = pd.to_timedelta(np.floor(nb_days)+1, unit="W")
            t_itv_before = pd.date_range(start=t_start[col][i] - shift, end=t_end[col][i] - shift, freq=p_freq)
            # If NaN is value to replace -> shift from one more week
            try:
                if np.isnan(np.sum(df_in[col][t_itv_before].values)):
                    shift_bef = shift
                    while np.isnan(np.sum(df_in[col][t_itv_before].values)):
                        shift_bef = shift_bef + pd.to_timedelta(1, unit="W")
                        if t_start[col][i] - shift_bef < df_in[col].index[0]:  # If the shift becomes out of bound
                            shift_before_break = True
                            break
                        t_itv_before = pd.date_range(start=t_start[col][i] - shift_bef, end=t_end[col][i] - shift_bef, freq=p_freq)
            except:
                error = True
                break
            t_itv_after = pd.date_range(start=t_start[col][i] + shift, end=t_end[col][i] + shift, freq=p_freq)
            try:
                if np.isnan(np.sum(df_in[col][t_itv_after].values)):
                    shift_after = shift
                    while np.isnan(np.sum(df_in[col][t_itv_after].values)):
                        shift_after = shift_after + pd.to_timedelta(1, unit="W")
                        if t_end[col][i] + shift_after > df_in[col].index[-1]:  # If the shift becomes out of bound
                            t_itv_after = t_itv_before
                            break
                        t_itv_after = pd.date_range(start=t_start[col][i] + shift_after, end=t_end[col][i] + shift_after, freq=p_freq)
            except:
                error = True
                break
            if shift_before_break:  # If shift was out of bound do:
                t_itv_before = t_itv_after
                break
            # Fill df_in with the average of the two
            if ~error:
                df_in[col][t_itv] = (df_in[col][t_itv_before].values + df_in[col][t_itv_after].values)/2
    return df_in


def rolling_window_fill(df_in, method='median', p_window=4):
    """"A rolling window median/mean filling function.
    p_window=2 - rolling window size (Should be less than 2 hours from C. Fan et al. 2015)
    method='median'/'mean' - choses the method to use for filling"""

    df_manip = df_in.copy()
    for col in df_manip:
    # calculate rolling median/mean
        if method == 'median':
            rm = df_manip[col].rolling(window=p_window, center=True).median()
        elif method == 'mean':
            rm = df_manip[col].rolling(window=p_window, center=True).mean()
        df_manip[col] = rm.ffill(limit=int(p_window/2+1)).bfill(limit=int(p_window/2+1)).interpolate(method='time', axis=0)
    return df_manip

## Data Cleaning
### Outlier detection
A Hampel filter moving window method is applied to detect point-wise outliers with a window size of 6 time-steps (hours) and a standard-deviation threshold of 3.
### Missing value filling
A single imputation approach is chosen in function of the length of the missing sections:
 - A moving average regression is chosen for missing data sections smaller than 3 consecutive hours,
 - Longer sections rely on the hot deck method, where missing values are averaged from identical time intervals and day of the week using sections of 2 weeks.

In [32]:
for f in meter_files:
    # Read
    meter_type = f.split("/")[10].split(".")[0]
    df = pd.read_csv(f, index_col="timestamp")
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')

    # Drop columns with more than 60% of missing values
    df = df.loc[:, df.isna().mean() < .6]

    # Outlier detection - Hampfel filter
    # for col in df_cub1.columns:
    #     df_cub1[col] = od.hampel_filter_pandas(df_cub1[col], window_size=4, n_sigmas=5) # Bug reported to pandas, with np.argwhere function, should be fixed in an upcoming patch

    # Missing value filling
    # Identifying missing value gaps within the dataframe
    t_start, t_end = cumulativemissing_id(df, p_freq='H', cumulative_threshold=4)
    # Hot-deck gap filling method
    df = gap_fill(df, t_start, t_end, p_freq='H')
    # Interpolate leftover gaps
    df = rolling_window_fill(df, method='mean', p_window=4)

    # Write results
    df.to_csv(path_data_out+meter_type+'.csv')